<a href="https://colab.research.google.com/github/hasini-06/RAG-Assignment/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers faiss-cpu

Ingestion

In [ ]:
sample_text="The Earth revolves around the Sun once every 365.25 days. This movement is responsible for the change in seasons."

Embedding Function

In [ ]:
from transformers import AutoModel,AutoTokenizer
import torch
import numpy as np

model_name="sentence-transformers/all-MiniLM-L6-v2"
tokenizer=AutoTokenizer.from_pretrained(model_name)   #Converts the context into tokens
model=AutoModel.from_pretrained(model_name)   # Model which helps to convert into tokens

def get_embedding(text):
  tokens=tokenizer(text,return_tensors="pt",truncation=True,padding=True)
  #truncation - cuts off the text if it's too long for the model, padding =  to make all sequences of the output in the same length

  with torch.no_grad():   #no_grad - to ask the model only to give answer but dont use this for its training
    output = model(**tokens)  #unpacking the tokens

  #takes last hidden states from the mean and take only single direction vector removing unnecessary dimensions and converting it into an array
  return output.last_hidden_state.mean(dim=1).squeeze().numpy()


In [ ]:
import faiss
chunks = [sample_text]    # storing context as list

embeddings = [get_embedding(chunk) for chunk in chunks]   # changing every word into tensors and storing it as a list


dim = len(embeddings[0])    # gives number of dimensions the first array has
index = faiss.IndexFlatL2(dim)    # calculate euclidean distance of each embedding vector
index.add(np.array(embeddings))   # adding embedded vectors into fiass index


In [ ]:
from transformers import pipeline
qa_pipepline = pipeline("text2text-generation",model = "google/flan-t5-small")

def retrieve_and_answer(query,top_k=1):   #top_k - takes the most similar result of all the all results
  query_embedding = get_embedding(query).reshape(1,-1)    # reshape - changes numpy array into 2D array
  _,indices = index.search(query_embedding,top_k)   # _,indeices - ignores distance and takes only indices, search - returns distances and indices
  retrieved_texts = [chunks[i] for i in indices[0]]   # uses retrieved indices to fetch the corresponding chunks and storing it in an array
  context = " ".join(retrieved_texts)   #joining the chunks into a context

  prompt = f"Context : {context} \n\n Question: {question} \nAnswer: "

  result = qa_pipepline(prompt,max_length=100)
  return result[0]['generated_text']


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
question=input("Enter the question : ")
answer=retrieve_and_answer(question)
print("Q : ",question)
print("Ans : ",answer)

Enter the question : Why do we have seasons on Earth?


Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Q :  Why do we have seasons on Earth?
Ans :  The Earth revolves around the Sun once every 365.25 days
